In [64]:
import pandas as pd
from pathlib import Path

data_folder = Path("..", "data")

yougov_folder = data_folder / "raw" / "yougov"

rows = {}

# iterate throug xlsx files in yougov folder
for file in yougov_folder.glob("*.xlsx"):
    print(file)
    # read xlsx file
    df = pd.read_excel(str(file))
    # set index to the first column
    df = df.set_index(df.columns[0]).transpose()

    # limit to just entries where the index starts with 2023- or 2024-
    df = df[df.index.str.startswith("2023-") | df.index.str.startswith("2024-")]
    df = df.drop(columns=["Unweighted base", "Base"])

    df.columns.name = file.stem

    favourable_columns = [
        col
        for col in df.columns
        if "favourable" in col.lower() and "unfavourable" not in col.lower()
    ]
    unfavourable_columns = [
        col
        for col in df.columns
        if "unfavourable" in col.lower() and " favourable" not in col.lower()
    ]

    df["favourable"] = df[favourable_columns].sum(axis=1)
    df["unfavourable"] = df[unfavourable_columns].sum(axis=1)
    df["net_favourable"] = df["favourable"] - df["unfavourable"]

    # delete unweighted base and base columns
    rows[file.stem] = df.mean()

../data/raw/yougov/how-brits-feel-about-the-oil-gas-mining-and-extraction-industries.xlsx
../data/raw/yougov/how-brits-feel-about-the-pharmaceutical-industry.xlsx
../data/raw/yougov/how-brits-feel-about-the-professional-services-industry-law-accounting-consulting-.xlsx
../data/raw/yougov/how-brits-feel-about-the-construction-industry.xlsx
../data/raw/yougov/how-brits-feel-about-utility-companies.xlsx
../data/raw/yougov/how-brits-feel-about-the-groceries-sector.xlsx
../data/raw/yougov/how-brits-feel-about-car-manufacturering-industry.xlsx
../data/raw/yougov/how-brits-feel-about-the-soft-drinks-industry.xlsx
../data/raw/yougov/how-brits-feel-about-the-military-industry.xlsx
../data/raw/yougov/how-brits-feel-about-the-tobacco-industry.xlsx
../data/raw/yougov/how-brits-feel-about-building-societies.xlsx
../data/raw/yougov/how-brits-feel-about-alcoholic-drink-makers.xlsx
../data/raw/yougov/how-brits-feel-about-technology-manufacturers.xlsx
../data/raw/yougov/how-brits-feel-about-the-gamblin

In [65]:
def fix_slug(slug: str) -> str:
    slug = slug.replace("how-brits-feel-about-", "")
    slug = slug.replace("-", " ")
    slug = slug.title()
    return slug


df = (
    pd.DataFrame(rows)
    .transpose()[
        [
            "favourable",
            "unfavourable",
            "Neither favourable nor unfavourable",
            "net_favourable",
        ]
    ]
    .reset_index()
)

df = df.rename(
    columns={
        "Neither favourable nor unfavourable": "neither_favourable_nor_unfavourable",
        "index": "industry",
    }
)
df = df.sort_values("net_favourable")
df["industry"] = df["industry"].apply(fix_slug)
df[["industry", "net_favourable"]]

,industry,net_favourable
9,The Tobacco Industry,-0.604444
13,The Gambling Industry,-0.561111
4,Utility Companies,-0.552778
0,The Oil Gas Mining And Extraction Industries,-0.402222
22,Media Companies,-0.371667
14,The Insurance Industry,-0.291667
18,The Luxury Fashion Industry,-0.212222
16,The Banking Industry,-0.137222
8,The Military Industry,-0.130000
15,Fast Food Chains,-0.106667
